In [1]:
import json
import tensorflow as tf
import numpy as np # linear algebra
from PIL import Image # image processing

In [2]:
TYPE = {0: 'MAJ', 1: 'MIN', 2: 'NBR'}
MAJ = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z'}
MIN = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z'}
NBR = {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9'}

In [3]:
def format_image(chemin: str) -> np.ndarray:
    img = Image.open(chemin) # 300 x 300
    
    nouvelle_taille = (25, 25) # 625
    
    img_redimensionnee = img.resize(nouvelle_taille) # Redimensionner l'image
    nb_image = img_redimensionnee.convert('L') # Convertion en noir et blanc
    
    tab = []
    for i in range(nb_image.size[1]):
        row = []
        for y in range(nb_image.size[0]):
            # Récupérer la couleur du pixel
            row.append(nb_image.getpixel((y, i)))
        tab.append(row)
        
    tab_numpy = np.array(tab) / 255 # mise à l'échelle des données
    tab_numpy_flatten = tab_numpy.flatten() # Transformation en tableau 1D
    print(len(tab_numpy_flatten))
    return tab_numpy_flatten

In [4]:
# Chargement du modèle sauvegardé
model_type = tf.keras.models.load_model("../models/Get_type_v4_b32_e10.keras")
model_maj = tf.keras.models.load_model("../models/Get_Maj_v4_b32_e10.keras")
model_min = tf.keras.models.load_model("../models/Get_Min_v4_b32_e10.keras")
model_nbr = tf.keras.models.load_model("../models/Get_Nbr_v4_b32_e10.keras")

In [59]:
def predict_image(chemin: str) -> json:
    """_summary_

    Args:
        chemin (str): chemin de l'image à prédire

    Returns:
        JSON: json contenant, le chemin de l'image, la prédiction finale et les prédictions des différents modèles
    """
    image_pixel = format_image(chemin) # Récupération des pixels de l'image
    image_pixel1d = image_pixel.reshape(1, 625) # Transformation en tableau 2D
    
    # Prédiction des différents modèles
    predictions_type = model_type.predict(image_pixel1d)
    predictions_maj = model_maj.predict(image_pixel1d)
    predictions_min = model_min.predict(image_pixel1d)
    predictions_nbr = model_nbr.predict(image_pixel1d)
    
    # Get max value of predictions
    max_type = max(predictions_type[0]) # Type
    max_maj = max(predictions_maj[0]) # Maj
    max_min = max(predictions_min[0]) # Min
    max_nbr = max(predictions_nbr[0]) # Nbr
    
    # Dictionnaire des probabilités
    dictionnaire = {
        max(predictions_maj[0]): "MAJ",
        max(predictions_min[0]): "MIN",
        max(predictions_nbr[0]): "NBR"
    }
    
    # Get label type
    type_img = TYPE[tf.argmax(predictions_type, axis=1).numpy()[0]] # label type
    
    # Get label forme
    maximum_forme = max(max_maj, max_min, max_nbr) # probabilité forme
    forme = dictionnaire[maximum_forme] # label type
    if forme == "MAJ":
        label_max_forme = MAJ[tf.argmax(predictions_maj, axis=1).numpy()[0]]
    elif forme == "MIN":
        label_max_forme = MIN[tf.argmax(predictions_min, axis=1).numpy()[0]]
    elif forme == "NBR":
        label_max_forme = NBR[tf.argmax(predictions_nbr, axis=1).numpy()[0]]
    
    # JSON de retour pour l'API
    json_retour_api = {
        "Image": chemin,
        "Label": "",
        "Maj": predictions_type[0][0],
        "Min": predictions_type[0][1],
        "Nbr": predictions_type[0][2],
        "A": predictions_maj[0][0],
        "B": predictions_maj[0][1],
        "C": predictions_maj[0][2],
        "D": predictions_maj[0][3],
        "E": predictions_maj[0][4],
        "F": predictions_maj[0][5],
        "G": predictions_maj[0][6],
        "H": predictions_maj[0][7],
        "I": predictions_maj[0][8],
        "J": predictions_maj[0][9],
        "K": predictions_maj[0][10],
        "L": predictions_maj[0][11],
        "M": predictions_maj[0][12],
        "N": predictions_maj[0][13],
        "O": predictions_maj[0][14],
        "P": predictions_maj[0][15],
        "Q": predictions_maj[0][16],
        "R": predictions_maj[0][17],
        "S": predictions_maj[0][18],
        "T": predictions_maj[0][19],
        "U": predictions_maj[0][20],
        "V": predictions_maj[0][21],
        "W": predictions_maj[0][22],
        "X": predictions_maj[0][23],
        "Y": predictions_maj[0][24],
        "Z": predictions_maj[0][25],
        "a": predictions_min[0][0],
        "b": predictions_min[0][1],
        "c": predictions_min[0][2],
        "d": predictions_min[0][3],
        "e": predictions_min[0][4],
        "f": predictions_min[0][5],
        "g": predictions_min[0][6],
        "h": predictions_min[0][7],
        "i": predictions_min[0][8],
        "j": predictions_min[0][9],
        "k": predictions_min[0][10],
        "l": predictions_min[0][11],
        "m": predictions_min[0][12],
        "n": predictions_min[0][13],
        "o": predictions_min[0][14],
        "p": predictions_min[0][15],
        "q": predictions_min[0][16],
        "r": predictions_min[0][17],
        "s": predictions_min[0][18],
        "t": predictions_min[0][19],
        "u": predictions_min[0][20],
        "v": predictions_min[0][21],
        "w": predictions_min[0][22],
        "x": predictions_min[0][23],
        "y": predictions_min[0][24],
        "z": predictions_min[0][25],
        "0": predictions_nbr[0][0],
        "1": predictions_nbr[0][1],
        "2": predictions_nbr[0][2],
        "3": predictions_nbr[0][3],
        "4": predictions_nbr[0][4],
        "5": predictions_nbr[0][5],
        "6": predictions_nbr[0][6],
        "7": predictions_nbr[0][7],
        "8": predictions_nbr[0][8],
        "9": predictions_nbr[0][9]
    }
    
    if type_img != forme:
        max_max_max = max(max_type, maximum_forme)
        if max_type == max_max_max:
            if type_img == "MAJ":
                json_retour_api["Label"] = MAJ[tf.argmax(predictions_maj, axis=1).numpy()[0]]
            elif type_img == "MIN":
                json_retour_api["Label"] = MIN[tf.argmax(predictions_min, axis=1).numpy()[0]]
            elif type_img == "NBR":
                json_retour_api["Label"] = NBR[tf.argmax(predictions_nbr, axis=1).numpy()[0]]
        else:
            json_retour_api["Label"] = label_max_forme
    else:
        json_retour_api["Label"] = label_max_forme
    
    return json_retour_api

In [61]:
chemin_image = "../../Dataset/dataset/train/e/002003.png"
json = predict_image(chemin_image)
json["Label"]

625
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


'5'